In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn

from load_data import RawDataLoader

os.environ["NFL_HOME"] = "/home/sam/repos/hobby-repos/nfl/"
rawLoader = RawDataLoader()
games_df, plays_df, players_df, location_data_df = rawLoader.get_data(weeks=[i for i in range(1, 10)])

# Exploring games_df

In [3]:
print(games_df.head())

       gameId  season  week   gameDate gameTimeEastern homeTeamAbbr  \
0  2022090800    2022     1   9/8/2022        20:20:00           LA   
1  2022091100    2022     1  9/11/2022        13:00:00          ATL   
2  2022091101    2022     1  9/11/2022        13:00:00          CAR   
3  2022091102    2022     1  9/11/2022        13:00:00          CHI   
4  2022091103    2022     1  9/11/2022        13:00:00          CIN   

  visitorTeamAbbr  homeFinalScore  visitorFinalScore  
0             BUF              10                 31  
1              NO              26                 27  
2             CLE              24                 26  
3              SF              19                 10  
4             PIT              20                 23  


In [8]:
print(f"Total games: {len(games_df)}")
home_wins = np.where(games_df['homeFinalScore'] > games_df['visitorFinalScore'], 1, 0).sum()
away_wins = np.where(games_df['visitorFinalScore'] > games_df['homeFinalScore'], 1, 0).sum()
ties = len(games_df) - home_wins - away_wins
print(f"Home team wins: {home_wins}")
print(f"Visitor team wins: {away_wins}")
print(f"Ties: {ties}")

Total games: 136
Home team wins: 73
Visitor team wins: 62
Ties: 1


In [10]:
unique_teams = pd.unique(games_df[['homeTeamAbbr', 'visitorTeamAbbr']].values.ravel())
print(unique_teams)

['LA' 'BUF' 'ATL' 'NO' 'CAR' 'CLE' 'CHI' 'SF' 'CIN' 'PIT' 'DET' 'PHI'
 'HOU' 'IND' 'MIA' 'NE' 'NYJ' 'BAL' 'WAS' 'JAX' 'ARI' 'KC' 'LAC' 'LV'
 'MIN' 'GB' 'TEN' 'NYG' 'DAL' 'TB' 'SEA' 'DEN']
